In [1]:
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.eos import calculate_eos
from ase.db import connect

db = connect('bulk.db')
for symb in ['Al', 'Ni', 'Cu', 'Pd', 'Ag', 'Pt', 'Au']:
    atoms = bulk(symb, 'fcc')
    atoms.calc = EMT()
    eos = calculate_eos(atoms)
    v, e, B = eos.fit()  # find minimum
    # Do one more calculation at the minimum and write to database:
    atoms.cell *= (v / atoms.get_volume())**(1 / 3)
    atoms.get_potential_energy()
    db.write(atoms, bm=B)

In [2]:
!ase db bulk.db -c +bm

id|age|user         |formula|calculator|energy|natoms| fmax|pbc|volume|charge|   mass|   bm
 1|29s|woodrowwilson|Al     |emt       |-0.005|     1|0.000|TTT|15.932| 0.000| 26.982|0.249
 2|29s|woodrowwilson|Ni     |emt       |-0.013|     1|0.000|TTT|10.601| 0.000| 58.693|1.105
 3|29s|woodrowwilson|Cu     |emt       |-0.007|     1|0.000|TTT|11.565| 0.000| 63.546|0.839
 4|28s|woodrowwilson|Pd     |emt       |-0.000|     1|0.000|TTT|14.588| 0.000|106.420|1.118
 5|28s|woodrowwilson|Ag     |emt       |-0.000|     1|0.000|TTT|16.775| 0.000|107.868|0.625
 6|28s|woodrowwilson|Pt     |emt       |-0.000|     1|0.000|TTT|15.080| 0.000|195.084|1.736
 7|28s|woodrowwilson|Au     |emt       |-0.000|     1|0.000|TTT|16.684| 0.000|196.967|1.085
Rows: 7
Keys: bm


In [3]:
!file bulk.db

bulk.db: SQLite 3.x database, last written using SQLite version 3038002, file counter 14, database pages 17, cookie 0xe, schema 4, UTF-8, version-valid-for 14


In [4]:
!ase db bulk.db --insert-into bulk.json

Added 0 key-value pairs (0 pairs updated)
Inserted 7 rows


In [5]:
!ls -lhtr

total 132K
-rw-rw-r-- 1 woodrowwilson woodrowwilson  48K Aug  2 10:08 ASE-CP2K.ipynb
-rw-r--r-- 1 woodrowwilson woodrowwilson  68K Aug  4 07:33 bulk.db
-rw-rw-r-- 1 woodrowwilson woodrowwilson 2.5K Aug  4 07:35 Databases.ipynb
-rw-rw-r-- 1 woodrowwilson woodrowwilson 5.2K Aug  4 07:36 bulk.json


In [7]:
from ase.calculators.emt import EMT
from ase.db import connect
from ase.build import fcc111, add_adsorbate
from ase.constraints import FixAtoms
from ase.optimize import BFGS

db1 = connect('bulk.db')
db2 = connect('ads.db')


def run(symb, a, n, ads):
    atoms = fcc111(symb, (1, 1, n), a=a)
    add_adsorbate(atoms, ads, height=1.0, position='fcc')

    # Constrain all atoms except the adsorbate:
    fixed = list(range(len(atoms) - 1))
    atoms.constraints = [FixAtoms(indices=fixed)]

    atoms.calc = EMT()
    opt = BFGS(atoms, logfile=None)
    opt.run(fmax=0.01)
    return atoms


for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        for ads in 'CNO':
            id = db2.reserve(layers=n, surf=symb, ads=ads)
            if id is not None:
                atoms = run(symb, a, n, ads)
                db2.write(atoms, layers=n, surf=symb, ads=ads)
                del db2[id]


In [8]:
!ase db ads.db natoms=0 -c ++


Rows: 0


In [9]:
!ase db ads.db natoms=0 --delete

Deleted 0 rows


In [10]:
from ase import Atoms
from ase.calculators.emt import EMT
from ase.db import connect
from ase.build import fcc111

db1 = connect('bulk.db')
db2 = connect('ads.db')


def run(symb, a, n):
    atoms = fcc111(symb, (1, 1, n), a=a)
    atoms.calc = EMT()
    atoms.get_forces()
    return atoms


# Clean slabs:
for row in db1.select():
    a = row.cell[0, 1] * 2
    symb = row.symbols[0]
    for n in [1, 2, 3]:
        id = db2.reserve(layers=n, surf=symb, ads='clean')
        if id is not None:
            atoms = run(symb, a, n)
            db2.write(atoms, id=id, layers=n, surf=symb, ads='clean')

# Atoms:
for ads in 'CNO':
    a = Atoms(ads)
    a.calc = EMT()
    a.get_potential_energy()
    db2.write(a)


In [11]:
!ase db ads.db -n

87 rows


In [12]:
!ase db ads.db ads=clean --insert-into refs.db

Added 0 key-value pairs (0 pairs updated)
Inserted 21 rows


In [13]:
!ase db ads.db ads=clean --delete --yes

Deleted 21 rows


In [14]:
!ase db ads.db pbc=FFF --insert-into refs.db

Added 0 key-value pairs (0 pairs updated)
Inserted 3 rows


In [15]:
!ase db ads.db pbc=FFF --delete --yes

Deleted 3 rows


In [16]:
!ase db ads.db -n

63 rows


In [17]:
!ase db refs.db -n

24 rows


In [18]:
from ase.db import connect

refs = connect('refs.db')
db = connect('ads.db')

for row in db.select():
    ea = (row.energy -
          refs.get(formula=row.ads).energy -
          refs.get(layers=row.layers, surf=row.surf).energy)
    h = row.positions[-1, 2] - row.positions[-2, 2]
    db.update(row.id, height=h, ea=ea)

In [25]:
!ase db ads.db layers=3 -c formula,ea,height

formula|    ea|height
Al3C   |-3.418| 1.262
Al3N   |-5.223| 1.219
Al3O   |-4.495| 1.553
Ni3C   |-3.730| 1.376
Ni3N   |-5.477| 1.382
Ni3O   |-4.741| 1.590
Cu3C   |-3.591| 1.378
Cu3N   |-5.344| 1.376
Cu3O   |-4.643| 1.587
Pd3C   |-3.630| 1.448
Pd3N   |-5.369| 1.461
Pd3O   |-4.660| 1.644
Ag3C   |-3.560| 1.377
Ag3N   |-5.336| 1.375
Ag3O   |-4.592| 1.599
Pt3C   |-3.715| 1.503
Pt3N   |-5.419| 1.535
Pt3O   |-4.724| 1.706
Au3C   |-3.547| 1.485
Au3N   |-5.281| 1.499
Rows: 21 (showing first 20)
Keys: ads, ea, height, layers, surf
